In [1]:
##install pubchem python library
!pip install pubchempy

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pubchempy as pcp
import pandas as pd

In [3]:
### Function to srap the data from PubChem
def cid_to_sol(id):
   x="https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/%s/xml"
   data=requests.get(x % id)#,timeout=(10,20) )
   html = BeautifulSoup(data.content, "xml")
   solubility = html.find(name='TOCHeading', string='Solubility')
   #print(dtype(solubility))
   if solubility == None:
      return None
   #elif          # or pass
   else:
     solub=solubility.find_next_sibling('Information').find(name='String').string
     return solub

In [4]:
df = pd.read_csv('pubchem_0k_10k.csv')

In [5]:
print(len(df))

10000


In [21]:
def cid_to_sol(id):
   x="https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/%s/xml"
   data=requests.get(x % id)#,timeout=(10,20) )
   html = BeautifulSoup(data.content, "xml")
   solubility = html.find(name='TOCHeading', string='Solubility')

   if solubility == None:
      return None

   else:
     solub=solubility.find_next_sibling('Information').find(name='String').string
     return solub
x=cid_to_sol(110635)
print(x)

None


In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from concurrent.futures import ThreadPoolExecutor

start = time.time()

df = pd.read_csv('pubchem_0k_10k.csv')
#df = pd.read_csv('/content/drive/MyDrive/pubchem_23000k_24000k.csv')
df_list = [df.iloc[i*1000:(i+1)*1000] for i in range(10)]

def cid_to_sol(id):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{id}/xml"
    try:
        data = requests.get(url)
        if data.status_code == 200:
            html = BeautifulSoup(data.content, "xml")
            solubility = html.find(name='TOCHeading', string='Solubility')

            if solubility is None:
                return None#'Data not found for this compound'
            else:
                solub = solubility.find_next_sibling('Information').find(name='String').string
                return solub
        else:
            return None#f'Error: Server returned status code {data.status_code}'
    except requests.exceptions.RequestException as e:
        return None#f'Error: Request Exception - {e}'

def process_chunk(df_chunk):
    P_sol = []
    for i in range(len(df_chunk)):
        try:
            time.sleep(0.6)  # Sleep for 0.6 seconds
            df_chunk = df_chunk.reset_index(drop=True)
            sol = cid_to_sol(df_chunk.CID[i])
            P_sol.append(sol)
        except AttributeError as e:
            sol = 'No string'
            P_sol.append(sol)
    return P_sol

p_sol_final = []

with ThreadPoolExecutor(max_workers=10) as executor:
    results = executor.map(process_chunk, df_list)
    for result in results:
        p_sol_final.append(result)

df1 = pd.DataFrame(p_sol_final)
joinlist = [x for sublist in p_sol_final for x in sublist]
df1 = pd.DataFrame(joinlist)
df = pd.concat(df_list)
df1 = df1.reset_index(drop=True)
df = df.reset_index(drop=True)
data = pd.concat([df, df1], axis=1, join='inner')
data.to_csv('pubchem_new_0_10k.csv',encoding='utf-8')


end = time.time()
duration = end - start
print(duration)

encoding error : input conversion failed due to input error, bytes 0x90 0x32 0x2C 0x34
encoding error : input conversion failed due to input error, bytes 0x90 0x32 0x2C 0x34
I/O error : encoder error
encoding error : input conversion failed due to input error, bytes 0x8F 0xC3 0x82 0x89
encoding error : input conversion failed due to input error, bytes 0x8F 0xC3 0x82 0x89
I/O error : encoder error
encoding error : input conversion failed due to input error, bytes 0x8E 0xC2 0xB1 0x2D
encoding error : input conversion failed due to input error, bytes 0x8E 0xC2 0xB1 0x2D
I/O error : encoder error
encoding error : input conversion failed due to input error, bytes 0x81 0xC2 0xB5 0x2C
encoding error : input conversion failed due to input error, bytes 0x81 0xC2 0xB5 0x2C
I/O error : encoder error
encoding error : input conversion failed due to input error, bytes 0x9D 0x29 0x20 0x63
encoding error : input conversion failed due to input error, bytes 0x9D 0x29 0x20 0x63
I/O error : encoder error


4446.9308269023895


In [18]:
final_data=pd.read_csv('pubchem_new_0_10000.csv')
final_data_raw=final_data[['CID','SMILES','0']]
final_data_raw.rename(columns = {'0':'Solubility'}, inplace = True)
final_raw = final_data_raw.loc[final_data_raw['Solubility'].notnull()]
print(len(final_raw))


1


/var/folders/k_/8s01c4dj0gnfx2n284nvm1t80000gn/T/ipykernel_13058/2306803033.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data_raw.rename(columns = {'0':'Solubility'}, inplace = True)


In [19]:
final_raw

,CID,SMILES,Solubility
2309,2466,CCCCOC1=CC=C(C=C1)CC(=O)NO,>33.5 [ug/mL] (The mean of the results at pH 7.4)


In [11]:
import aiohttp

In [17]:
import pandas as pd
import aiohttp
import asyncio
import time
from bs4 import BeautifulSoup
import chardet  # For detecting file encoding

async def fetch_data(session, id, semaphore):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{id}/xml"
    retry_count = 3
    for _ in range(retry_count):
        async with semaphore:
            try:
                async with session.get(url, timeout=120) as response:
                    return await response.text()
            except (asyncio.TimeoutError, aiohttp.ClientError) as e:
                print(f"Error fetching data for ID: {id}. Retrying...")
    return None

async def cid_to_sol(id, semaphore):
    async with aiohttp.ClientSession() as session:
        xml_content = await fetch_data(session, id, semaphore)

    if xml_content is None:
        return None

    html = BeautifulSoup(xml_content, "xml")
    solubility = html.find(name='TOCHeading', string='Solubility')

    if solubility is None:
        return None
    else:
        information = solubility.find_next_sibling('Information')
        if information:
            solub = information.find(name='String')
            if solub:
                return solub.string
        return None

async def process_cids(start_idx, end_idx):
    # Detect file encoding
    with open('pubchem_0k_10k.csv', 'rb') as f:
        result = chardet.detect(f.read())
        encoding = result['encoding']

    try:
        # Read CSV file with detected encoding
        df = pd.read_csv('pubchem_0k_10k.csv', encoding=encoding)
    except UnicodeDecodeError:
        # Retry reading the file with 'utf-8' encoding
        df = pd.read_csv('pubchem_0k_10k.csv', encoding='utf-8')

    df_slice = df.iloc[start_idx:end_idx]

    start = time.time()

    p_sol_final = []

    # Limit concurrent requests with a semaphore
    semaphore = asyncio.Semaphore(5)  # Limit to 5 concurrent requests

    for chunk in [df_slice.iloc[i:i+1000] for i in range(0, len(df_slice), 1000)]:
        sols = await asyncio.gather(*(cid_to_sol(cid, semaphore) for cid in chunk['CID']))
        p_sol_final.extend(sols)

        # Introduce a delay to comply with the rate limit (5 requests per second)
        await asyncio.sleep(1)

    df1 = pd.DataFrame(p_sol_final)
    df_slice = df_slice.reset_index(drop=True)

    file_name = f'pubchem_new_{start_idx}_{end_idx}.csv'
    data = pd.concat([df_slice, df1], axis=1, join='inner')

    data.to_csv(file_name, index=False)

    end = time.time()
    duration = end - start
    print(f"Processed CID from {start_idx} to {end_idx} - Total execution time: {duration} seconds")

async def main():
    await process_cids(0, 10000)

# Run the asynchronous function
await main()


Processed CID from 0 to 10000 - Total execution time: 959.2631621360779 seconds


In [ ]:
import pandas as pd
!pip install pubchempy
import requests
from bs4 import BeautifulSoup
import re
import pubchempy as pcp
import pandas as pd
df2=df.iloc[200000:210000]
dfnew1=df2[0:1000]
dfnew2=df2[1000:2000]
dfnew3=df2[2000:3000]
dfnew4=df2[3000:4000]
dfnew5=df2[4000:5000]
dfnew6=df2[5000:6000]
dfnew7=df2[6000:7000]
dfnew8=df2[7000:8000]
dfnew9=df2[8000:9000]
dfnew10=df2[9000:10000]
import time
start = time.time()
#dfnew1 = dfnew1.reset_index()
obj=[dfnew1, dfnew2, dfnew3,dfnew4, dfnew5,dfnew6, dfnew7,dfnew8, dfnew9, dfnew10]
#obj= [dfnew11, dfnew12, dfnew13,dfnew14, dfnew15, dfnew16,dfnew17, dfnew18, dfnew19,dfnew20]
#dfnew1 = dfnew1.reset_index()
p_sol_final=[]
for j in range(10):
    P_sol=[]
    for i in range(1000):
      try:
        time.sleep(0.2) # Sleep for 1 seconds
        obj[j] = obj[j].reset_index(drop=True)
        sol=cid_to_sol(obj[j].CID[i])
        #
        P_sol.append(sol)
      except AttributeError as e:
        sol=='No string'
        P_sol.append(sol)
    p_sol_final.append(P_sol)
df1 = pd.DataFrame(p_sol_final)
joinlist = [x for list in p_sol_final for x in list]
df1 = pd.DataFrame(joinlist)
df=pd.concat(obj)
df1 = df1.reset_index(drop=True)
df = df.reset_index(drop=True)
data = pd.concat([df, df1], axis=1, join='inner')
data.to_csv('/content/drive/MyDrive/pubchem_20100000_20210000.csv')
end = time.time()
duration = end-start
print(duration)

3789.4090712070465


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/pubchem_20000k_21000k')
!pip install pubchempy
import requests
from bs4 import BeautifulSoup
import re
import pubchempy as pcp
import pandas as pd
def cid_to_sol(id):
   x="https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/%s/xml"
   data=requests.get(x % id)#,timeout=(10,20) )
   html = BeautifulSoup(data.content, "xml")
   solubility = html.find(name='TOCHeading', string='Solubility')
   #print(dtype(solubility))
   if solubility == None:
      return None
   #elif          # or pass
   else:
     solub=solubility.find_next_sibling('Information').find(name='String').string
     return solub
df2=df.iloc[210000:220000]
dfnew1=df2[0:1000]
dfnew2=df2[1000:2000]
dfnew3=df2[2000:3000]
dfnew4=df2[3000:4000]
dfnew5=df2[4000:5000]
dfnew6=df2[5000:6000]
dfnew7=df2[6000:7000]
dfnew8=df2[7000:8000]
dfnew9=df2[8000:9000]
dfnew10=df2[9000:10000]
import time
start = time.time()
#dfnew1 = dfnew1.reset_index()
obj=[dfnew1, dfnew2, dfnew3,dfnew4, dfnew5,dfnew6, dfnew7,dfnew8, dfnew9, dfnew10]
#obj= [dfnew11, dfnew12, dfnew13,dfnew14, dfnew15, dfnew16,dfnew17, dfnew18, dfnew19,dfnew20]
#dfnew1 = dfnew1.reset_index()
p_sol_final=[]
for j in range(10):
    P_sol=[]
    for i in range(1000):
      try:
        time.sleep(0.2) # Sleep for 1 seconds
        obj[j] = obj[j].reset_index(drop=True)
        sol=cid_to_sol(obj[j].CID[i])
        #
        P_sol.append(sol)
      except AttributeError as e:
        sol=='No string'
        P_sol.append(sol)
    p_sol_final.append(P_sol)
df1 = pd.DataFrame(p_sol_final)
joinlist = [x for list in p_sol_final for x in list]
df1 = pd.DataFrame(joinlist)
df=pd.concat(obj)
df1 = df1.reset_index(drop=True)
df = df.reset_index(drop=True)
data = pd.concat([df, df1], axis=1, join='inner')
data.to_csv('/content/drive/MyDrive/pubchem_20210000_20220000.csv')
end = time.time()
duration = end-start
print(duration)

6074.94059920311


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/pubchem_20000k_21000k')
!pip install pubchempy
import requests
from bs4 import BeautifulSoup
import re
import pubchempy as pcp
import pandas as pd
def cid_to_sol(id):
   x="https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/%s/xml"
   data=requests.get(x % id)#,timeout=(10,20) )
   html = BeautifulSoup(data.content, "xml")
   solubility = html.find(name='TOCHeading', string='Solubility')
   #print(dtype(solubility))
   if solubility == None:
      return None
   #elif          # or pass
   else:
     solub=solubility.find_next_sibling('Information').find(name='String').string
     return solub
df2=df.iloc[220000:230000]
dfnew1=df2[0:1000]
dfnew2=df2[1000:2000]
dfnew3=df2[2000:3000]
dfnew4=df2[3000:4000]
dfnew5=df2[4000:5000]
dfnew6=df2[5000:6000]
dfnew7=df2[6000:7000]
dfnew8=df2[7000:8000]
dfnew9=df2[8000:9000]
dfnew10=df2[9000:10000]
import time
start = time.time()
#dfnew1 = dfnew1.reset_index()
obj=[dfnew1, dfnew2, dfnew3,dfnew4, dfnew5,dfnew6, dfnew7,dfnew8, dfnew9, dfnew10]
#obj= [dfnew11, dfnew12, dfnew13,dfnew14, dfnew15, dfnew16,dfnew17, dfnew18, dfnew19,dfnew20]
#dfnew1 = dfnew1.reset_index()
p_sol_final=[]
for j in range(10):
    P_sol=[]
    for i in range(1000):
      try:
        time.sleep(0.2) # Sleep for 1 seconds
        obj[j] = obj[j].reset_index(drop=True)
        sol=cid_to_sol(obj[j].CID[i])
        #
        P_sol.append(sol)
      except AttributeError as e:
        sol=='No string'
        P_sol.append(sol)
    p_sol_final.append(P_sol)
df1 = pd.DataFrame(p_sol_final)
joinlist = [x for list in p_sol_final for x in list]
df1 = pd.DataFrame(joinlist)
df=pd.concat(obj)
df1 = df1.reset_index(drop=True)
df = df.reset_index(drop=True)
data = pd.concat([df, df1], axis=1, join='inner')
data.to_csv('/content/drive/MyDrive/pubchem_20220000_20230000.csv')
end = time.time()
duration = end-start
print(duration)

6066.656909942627


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/pubchem_20260000_20270000.csv')

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/pubchem_20000k_21000k')
!pip install pubchempy
import requests
from bs4 import BeautifulSoup
import re
import pubchempy as pcp
import pandas as pd
def cid_to_sol(id):
   x="https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/%s/xml"
   data=requests.get(x % id)#,timeout=(10,20) )
   html = BeautifulSoup(data.content, "xml")
   solubility = html.find(name='TOCHeading', string='Solubility')
   #print(dtype(solubility))
   if solubility == None:
      return None
   #elif          # or pass
   else:
     solub=solubility.find_next_sibling('Information').find(name='String').string
     return solub
df2=df.iloc[230000:240000]
dfnew1=df2[0:1000]
dfnew2=df2[1000:2000]
dfnew3=df2[2000:3000]
dfnew4=df2[3000:4000]
dfnew5=df2[4000:5000]
dfnew6=df2[5000:6000]
dfnew7=df2[6000:7000]
dfnew8=df2[7000:8000]
dfnew9=df2[8000:9000]
dfnew10=df2[9000:10000]
import time
start = time.time()
#dfnew1 = dfnew1.reset_index()
obj=[dfnew1, dfnew2, dfnew3,dfnew4, dfnew5,dfnew6, dfnew7,dfnew8, dfnew9, dfnew10]
#obj= [dfnew11, dfnew12, dfnew13,dfnew14, dfnew15, dfnew16,dfnew17, dfnew18, dfnew19,dfnew20]
#dfnew1 = dfnew1.reset_index()
p_sol_final=[]
for j in range(10):
    P_sol=[]
    for i in range(1000):
      try:
        time.sleep(0.2) # Sleep for 1 seconds
        obj[j] = obj[j].reset_index(drop=True)
        sol=cid_to_sol(obj[j].CID[i])
        #
        P_sol.append(sol)
      except AttributeError as e:
        sol=='No string'
        P_sol.append(sol)
    p_sol_final.append(P_sol)
df1 = pd.DataFrame(p_sol_final)
joinlist = [x for list in p_sol_final for x in list]
df1 = pd.DataFrame(joinlist)
df=pd.concat(obj)
df1 = df1.reset_index(drop=True)
df = df.reset_index(drop=True)
data = pd.concat([df, df1], axis=1, join='inner')
data.to_csv('/content/drive/MyDrive/pubchem_20230000_20240000.csv')
end = time.time()
duration = end-start
print(duration)

4213.850644350052


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/pubchem_20000k_21000k')
!pip install pubchempy
import requests
from bs4 import BeautifulSoup
import re
import pubchempy as pcp
import pandas as pd
def cid_to_sol(id):
   x="https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/%s/xml"
   data=requests.get(x % id)#,timeout=(10,20) )
   html = BeautifulSoup(data.content, "xml")
   solubility = html.find(name='TOCHeading', string='Solubility')
   #print(dtype(solubility))
   if solubility == None:
      return None
   #elif          # or pass
   else:
     solub=solubility.find_next_sibling('Information').find(name='String').string
     return solub
df2=df.iloc[240000:250000]
dfnew1=df2[0:1000]
dfnew2=df2[1000:2000]
dfnew3=df2[2000:3000]
dfnew4=df2[3000:4000]
dfnew5=df2[4000:5000]
dfnew6=df2[5000:6000]
dfnew7=df2[6000:7000]
dfnew8=df2[7000:8000]
dfnew9=df2[8000:9000]
dfnew10=df2[9000:10000]
import time
start = time.time()
#dfnew1 = dfnew1.reset_index()
obj=[dfnew1, dfnew2, dfnew3,dfnew4, dfnew5,dfnew6, dfnew7,dfnew8, dfnew9, dfnew10]
#obj= [dfnew11, dfnew12, dfnew13,dfnew14, dfnew15, dfnew16,dfnew17, dfnew18, dfnew19,dfnew20]
#dfnew1 = dfnew1.reset_index()
p_sol_final=[]
for j in range(10):
    P_sol=[]
    for i in range(1000):
      try:
        time.sleep(0.2) # Sleep for 1 seconds
        obj[j] = obj[j].reset_index(drop=True)
        sol=cid_to_sol(obj[j].CID[i])
        #
        P_sol.append(sol)
      except AttributeError as e:
        sol=='No string'
        P_sol.append(sol)
    p_sol_final.append(P_sol)
df1 = pd.DataFrame(p_sol_final)
joinlist = [x for list in p_sol_final for x in list]
df1 = pd.DataFrame(joinlist)
df=pd.concat(obj)
df1 = df1.reset_index(drop=True)
df = df.reset_index(drop=True)
data = pd.concat([df, df1], axis=1, join='inner')
data.to_csv('/content/drive/MyDrive/pubchem_20240000_20250000.csv')
end = time.time()
duration = end-start
print(duration)

4268.814210176468


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/pubchem_20000k_21000k')
!pip install pubchempy
import requests
from bs4 import BeautifulSoup
import re
import pubchempy as pcp
import pandas as pd
def cid_to_sol(id):
   x="https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/%s/xml"
   data=requests.get(x % id)#,timeout=(10,20) )
   html = BeautifulSoup(data.content, "xml")
   solubility = html.find(name='TOCHeading', string='Solubility')
   #print(dtype(solubility))
   if solubility == None:
      return None
   #elif          # or pass
   else:
     solub=solubility.find_next_sibling('Information').find(name='String').string
     return solub
df2=df.iloc[250000:260000]
dfnew1=df2[0:1000]
dfnew2=df2[1000:2000]
dfnew3=df2[2000:3000]
dfnew4=df2[3000:4000]
dfnew5=df2[4000:5000]
dfnew6=df2[5000:6000]
dfnew7=df2[6000:7000]
dfnew8=df2[7000:8000]
dfnew9=df2[8000:9000]
dfnew10=df2[9000:10000]
import time
start = time.time()
#dfnew1 = dfnew1.reset_index()
obj=[dfnew1, dfnew2, dfnew3,dfnew4, dfnew5,dfnew6, dfnew7,dfnew8, dfnew9, dfnew10]
#obj= [dfnew11, dfnew12, dfnew13,dfnew14, dfnew15, dfnew16,dfnew17, dfnew18, dfnew19,dfnew20]
#dfnew1 = dfnew1.reset_index()
p_sol_final=[]
for j in range(10):
    P_sol=[]
    for i in range(1000):
      try:
        time.sleep(0.2) # Sleep for 1 seconds
        obj[j] = obj[j].reset_index(drop=True)
        sol=cid_to_sol(obj[j].CID[i])
        #
        P_sol.append(sol)
      except AttributeError as e:
        sol=='No string'
        P_sol.append(sol)
    p_sol_final.append(P_sol)
df1 = pd.DataFrame(p_sol_final)
joinlist = [x for list in p_sol_final for x in list]
df1 = pd.DataFrame(joinlist)
df=pd.concat(obj)
df1 = df1.reset_index(drop=True)
df = df.reset_index(drop=True)
data = pd.concat([df, df1], axis=1, join='inner')
data.to_csv('/content/drive/MyDrive/pubchem_20250000_20260000.csv')
end = time.time()
duration = end-start
print(duration)

4284.045051574707


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/pubchem_20000k_21000k')
!pip install pubchempy
import requests
from bs4 import BeautifulSoup
import re
import pubchempy as pcp
import pandas as pd
def cid_to_sol(id):
   x="https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/%s/xml"
   data=requests.get(x % id)#,timeout=(10,20) )
   html = BeautifulSoup(data.content, "xml")
   solubility = html.find(name='TOCHeading', string='Solubility')
   #print(dtype(solubility))
   if solubility == None:
      return None
   #elif          # or pass
   else:
     solub=solubility.find_next_sibling('Information').find(name='String').string
     return solub
df2=df.iloc[260000:270000]
dfnew1=df2[0:1000]
dfnew2=df2[1000:2000]
dfnew3=df2[2000:3000]
dfnew4=df2[3000:4000]
dfnew5=df2[4000:5000]
dfnew6=df2[5000:6000]
dfnew7=df2[6000:7000]
dfnew8=df2[7000:8000]
dfnew9=df2[8000:9000]
dfnew10=df2[9000:10000]
import time
start = time.time()
#dfnew1 = dfnew1.reset_index()
obj=[dfnew1, dfnew2, dfnew3,dfnew4, dfnew5,dfnew6, dfnew7,dfnew8, dfnew9, dfnew10]
#obj= [dfnew11, dfnew12, dfnew13,dfnew14, dfnew15, dfnew16,dfnew17, dfnew18, dfnew19,dfnew20]
#dfnew1 = dfnew1.reset_index()
p_sol_final=[]
for j in range(10):
    P_sol=[]
    for i in range(1000):
      try:
        time.sleep(0.2) # Sleep for 1 seconds
        obj[j] = obj[j].reset_index(drop=True)
        sol=cid_to_sol(obj[j].CID[i])
        #
        P_sol.append(sol)
      except AttributeError as e:
        sol=='No string'
        P_sol.append(sol)
    p_sol_final.append(P_sol)
df1 = pd.DataFrame(p_sol_final)
joinlist = [x for list in p_sol_final for x in list]
df1 = pd.DataFrame(joinlist)
df=pd.concat(obj)
df1 = df1.reset_index(drop=True)
df = df.reset_index(drop=True)
data = pd.concat([df, df1], axis=1, join='inner')
data.to_csv('/content/drive/MyDrive/pubchem_20260000_20270000.csv')
end = time.time()
duration = end-start
print(duration)

4313.86141204834


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/pubchem_20000k_21000k')
!pip install pubchempy
import requests
from bs4 import BeautifulSoup
import re
import pubchempy as pcp
import pandas as pd
def cid_to_sol(id):
   x="https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/%s/xml"
   data=requests.get(x % id)#,timeout=(10,20) )
   html = BeautifulSoup(data.content, "xml")
   solubility = html.find(name='TOCHeading', string='Solubility')
   #print(dtype(solubility))
   if solubility == None:
      return None
   #elif          # or pass
   else:
     solub=solubility.find_next_sibling('Information').find(name='String').string
     return solub
df2=df.iloc[270000:280000]
dfnew1=df2[0:1000]
dfnew2=df2[1000:2000]
dfnew3=df2[2000:3000]
dfnew4=df2[3000:4000]
dfnew5=df2[4000:5000]
dfnew6=df2[5000:6000]
dfnew7=df2[6000:7000]
dfnew8=df2[7000:8000]
dfnew9=df2[8000:9000]
dfnew10=df2[9000:10000]
import time
start = time.time()
#dfnew1 = dfnew1.reset_index()
obj=[dfnew1, dfnew2, dfnew3,dfnew4, dfnew5,dfnew6, dfnew7,dfnew8, dfnew9, dfnew10]
#obj= [dfnew11, dfnew12, dfnew13,dfnew14, dfnew15, dfnew16,dfnew17, dfnew18, dfnew19,dfnew20]
#dfnew1 = dfnew1.reset_index()
p_sol_final=[]
for j in range(10):
    P_sol=[]
    for i in range(1000):
      try:
        time.sleep(0.2) # Sleep for 1 seconds
        obj[j] = obj[j].reset_index(drop=True)
        sol=cid_to_sol(obj[j].CID[i])
        #
        P_sol.append(sol)
      except AttributeError as e:
        sol=='No string'
        P_sol.append(sol)
    p_sol_final.append(P_sol)
df1 = pd.DataFrame(p_sol_final)
joinlist = [x for list in p_sol_final for x in list]
df1 = pd.DataFrame(joinlist)
df=pd.concat(obj)
df1 = df1.reset_index(drop=True)
df = df.reset_index(drop=True)
data = pd.concat([df, df1], axis=1, join='inner')
data.to_csv('/content/drive/MyDrive/pubchem_20270000_20280000.csv')
end = time.time()
duration = end-start
print(duration)

10874.484139680862


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/pubchem_20000k_21000k')
!pip install pubchempy
import requests
from bs4 import BeautifulSoup
import re
import pubchempy as pcp
import pandas as pd
def cid_to_sol(id):
   x="https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/%s/xml"
   data=requests.get(x % id)#,timeout=(10,20) )
   html = BeautifulSoup(data.content, "xml")
   solubility = html.find(name='TOCHeading', string='Solubility')
   #print(dtype(solubility))
   if solubility == None:
      return None
   #elif          # or pass
   else:
     solub=solubility.find_next_sibling('Information').find(name='String').string
     return solub
df2=df.iloc[280000:290000]
dfnew1=df2[0:1000]
dfnew2=df2[1000:2000]
dfnew3=df2[2000:3000]
dfnew4=df2[3000:4000]
dfnew5=df2[4000:5000]
dfnew6=df2[5000:6000]
dfnew7=df2[6000:7000]
dfnew8=df2[7000:8000]
dfnew9=df2[8000:9000]
dfnew10=df2[9000:10000]
import time
start = time.time()
#dfnew1 = dfnew1.reset_index()
obj=[dfnew1, dfnew2, dfnew3,dfnew4, dfnew5,dfnew6, dfnew7,dfnew8, dfnew9, dfnew10]
#obj= [dfnew11, dfnew12, dfnew13,dfnew14, dfnew15, dfnew16,dfnew17, dfnew18, dfnew19,dfnew20]
#dfnew1 = dfnew1.reset_index()
p_sol_final=[]
for j in range(10):
    P_sol=[]
    for i in range(1000):
      try:
        time.sleep(0.2) # Sleep for 1 seconds
        obj[j] = obj[j].reset_index(drop=True)
        sol=cid_to_sol(obj[j].CID[i])
        #
        P_sol.append(sol)
      except AttributeError as e:
        sol=='No string'
        P_sol.append(sol)
    p_sol_final.append(P_sol)
df1 = pd.DataFrame(p_sol_final)
joinlist = [x for list in p_sol_final for x in list]
df1 = pd.DataFrame(joinlist)
df=pd.concat(obj)
df1 = df1.reset_index(drop=True)
df = df.reset_index(drop=True)
data = pd.concat([df, df1], axis=1, join='inner')
data.to_csv('/content/drive/MyDrive/pubchem_20280000_20290000.csv')
end = time.time()
duration = end-start
print(duration)

10799.550745725632


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/pubchem_20000k_21000k')
!pip install pubchempy
import requests
from bs4 import BeautifulSoup
import re
import pubchempy as pcp
import pandas as pd
def cid_to_sol(id):
   x="https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/%s/xml"
   data=requests.get(x % id)#,timeout=(10,20) )
   html = BeautifulSoup(data.content, "xml")
   solubility = html.find(name='TOCHeading', string='Solubility')
   #print(dtype(solubility))
   if solubility == None:
      return None
   #elif          # or pass
   else:
     solub=solubility.find_next_sibling('Information').find(name='String').string
     return solub
df2=df.iloc[290000:300000]
dfnew1=df2[0:1000]
dfnew2=df2[1000:2000]
dfnew3=df2[2000:3000]
dfnew4=df2[3000:4000]
dfnew5=df2[4000:5000]
dfnew6=df2[5000:6000]
dfnew7=df2[6000:7000]
dfnew8=df2[7000:8000]
dfnew9=df2[8000:9000]
dfnew10=df2[9000:10000]
import time
start = time.time()
#dfnew1 = dfnew1.reset_index()
obj=[dfnew1, dfnew2, dfnew3,dfnew4, dfnew5,dfnew6, dfnew7,dfnew8, dfnew9, dfnew10]
#obj= [dfnew11, dfnew12, dfnew13,dfnew14, dfnew15, dfnew16,dfnew17, dfnew18, dfnew19,dfnew20]
#dfnew1 = dfnew1.reset_index()
p_sol_final=[]
for j in range(10):
    P_sol=[]
    for i in range(1000):
      try:
        time.sleep(0.2) # Sleep for 1 seconds
        obj[j] = obj[j].reset_index(drop=True)
        sol=cid_to_sol(obj[j].CID[i])
        #
        P_sol.append(sol)
      except AttributeError as e:
        sol=='No string'
        P_sol.append(sol)
    p_sol_final.append(P_sol)
df1 = pd.DataFrame(p_sol_final)
joinlist = [x for list in p_sol_final for x in list]
df1 = pd.DataFrame(joinlist)
df=pd.concat(obj)
df1 = df1.reset_index(drop=True)
df = df.reset_index(drop=True)
data = pd.concat([df, df1], axis=1, join='inner')
data.to_csv('/content/drive/MyDrive/pubchem_20290000_20300000.csv')
end = time.time()
duration = end-start
print(duration)